In [4]:
import os
import besos
from besos import sampling
from besos import eppy_funcs as ef
from besos import eplus_funcs as ep
from besos.problem import EPProblem
from besos.evaluator import EvaluatorEP
from besos.parameters import wwr, RangeParameter, FieldSelector, FilterSelector, GenericSelector, Parameter, expand_plist, DependentParameter, CategoryParameter
from besos.objectives import MeterReader,time_series_values,EPResults
from besos.optimizer import NSGAII, df_solution_to_solutions
import pandas as pd
import matplotlib.pyplot as plt
from seaborn import heatmap
from seaborn import pairplot
import numpy as np

# Open the IDF file
idf_1 = "/home/ict4bd/Desktop/ict4building/DesignBuilderModel/1_layer_window.idf"
idf_2 = "/home/ict4bd/Desktop/ict4building/DesignBuilderModel/2_layer_window.idf"
idf_3 = "/home/ict4bd/Desktop/ict4building/DesignBuilderModel/3_layer_window.idf"
weather_file = '/home/ict4bd/Desktop/ict4building/Requirements/CHE_Geneva.067000_IWEC/CHE_Geneva.067000_IWEC.epw'
## Set up evaluator
# Load Building
building = ef.get_building(idf_3)

# Parameter Space
parameters = []

orientation = FieldSelector(class_name='Building', field_name='North Axis')
orientation_descriptor = CategoryParameter(options=[0, 90, 180, 270])

insulation = FieldSelector(class_name='Material', object_name='XPS Extruded Polystyrene  - CO2 Blowing_.O792', 
                           field_name='Thickness')
insulation_descriptor = RangeParameter(min_val = 0.01, max_val=0.5) 

shading = FieldSelector(class_name='WindowShadingControl',field_name='Setpoint')
shading_descriptor = RangeParameter(min_val = 0, max_val=30) 

ventilation = FieldSelector(class_name='ZoneVentilation:DesignFlowRate',field_name='Design Flow Rate')
ventilation_descriptor = CategoryParameter(options=[0, 0.12])




parameters.append(Parameter(selector=orientation, 
                              value_descriptor=orientation_descriptor,
                              name='Orientation'))

parameters.append(Parameter(selector=ventilation,
                            value_descriptor=ventilation_descriptor,
                            name='Ventilation'))

parameters.append(Parameter(selector=shading,
                            value_descriptor=shading_descriptor,
                            name='Shading'))

parameters.append(Parameter(selector=insulation,
                            value_descriptor=insulation_descriptor,
                            name='Insulation Thickness'))

parameters.append(wwr(RangeParameter(0.1,0.9))) 

# Objectives of our problem which is to minimize the TOTAL COMSUMPTION
objectives = ['Electricity:Facility','DistrictCooling:Facility', 'DistrictHeating:Facility']

# problem = parameters + objectives
besos_problem = EPProblem(parameters, objectives)

# EPProblem vs EvaluatorEP = problem + building + weather
evaluator = EvaluatorEP(besos_problem, building, out_dir='outputdir_reg', err_dir='outputdir_reg', epw_file=weather_file) 

samples_temp = []
'''
orientation_values = [0,90,180,270] 
wwr_values = [.15, .5, .9] 
ventilation_values = [0, 0.12] 
tickness_values = np.linspace(0.01,0.5,5)
shading = np.linspace(5,25,6)
for ori in orientation_values:
    for wwr_ in wwr_values:
        for vent in ventilation_values:
            for tickness in tickness_values:
                for shade in shading:
                    samples_temp.append({'Orientation': ori,
                                         'Ventilation': vent,
                                         'Shading': shade,
                                         'Insulation Thickness': tickness,
                                         'Window to Wall Ratio': wwr_,})
'''
samples_temp.append({'Orientation': 270,
                     'Ventilation': 0,
                     'Shading': 0,
                     'Insulation Thickness': 0.5,
                     'Window to Wall Ratio': 0.1,})
samples = pd.DataFrame.from_dict(samples_temp)
samples

/home/ict4bd/py3/lib/python3.8/site-packages/besos/evaluator.py:659: FutureWarning: epw_file has been deprecated and will be removed in the future. Use epw instead.
  warn(


,Orientation,Ventilation,Shading,Insulation Thickness,Window to Wall Ratio
0,270,0,0,0.5,0.1


In [5]:
# return dataframe with results described in objectives
outputs = evaluator.df_apply(samples, keep_input=True)

In [6]:
# Q is kWh / m2
total_area= 0
for Zones in building.idfobjects["Zone"]:
    total_area += float(Zones.checkrange("Floor_Area")) 
outputs['total (Q)'] = outputs[objectives].sum(axis=1) /3.6e6 /total_area 
outputs[outputs['total (Q)']==outputs['total (Q)'].min()]

,Orientation,Ventilation,Shading,Insulation Thickness,Window to Wall Ratio,Electricity:Facility,DistrictCooling:Facility,DistrictHeating:Facility,total (Q)
0,270,0,0,0.5,0.1,5.773012e+09,0.0,1.308721e+10,111.790961
